In [24]:
import os

In [1]:
import random
import tensorflow as tf
import numpy as np

In [2]:
symbol = {
    '+':1,
    '-':2,
    '*':3,
    '/':4,
}

symbol_s = {
    1:'+',
    2:'-',
    3:'*',
    4:'/',
}

In [3]:
upper_bound=10
lower_bound=1

def generateEquation():
    eq = []
    eq.append(str(random.randint(lower_bound, upper_bound)))
    for i in range(4):
        eq.append(symbol_s[random.randint(1, 4)*1])
        eq.append(str(random.randint(lower_bound, upper_bound)))
    return eq

In [4]:
test = generateEquation()
print(''.join(test))

8/9+7/5*3


In [5]:
def generateStep(a):  
    mark=[] # step1 marks
    sol = [a]
    
    n = len(a)//2
    
    prev = a
    for i in range(n-1,-1,-1):
        x = random.randint(0,i)*2
        #print("i="+str(i)+"x="+str(x))
        #print(prev[:2*x],prev[2*x+3:])
        
        middle = x+1
        while prev[middle]=='':
            middle+=1;
        
        start = middle-1
        while prev[start]=='':
            start-=1;
            
        end = middle+1
        while prev[end]=='':
            end+=1;
        #print(start,middle,end)
        
        #prefix
        step = prev[:start]
        #process step
        
        cal_value = eval(''.join(prev[start:end+1]))
        step.append(str(cal_value))
        #suffix
        step += ['']*(end-start)
        step += prev[end+1:]
        sol.append(step)
        
        #print(step)
        #print(''.join(prev[x:x+3]))
        
        #give mark
        sym = a[middle]
        if sym == '+':
            mark.append(0) if (('*' in prev) or ('/' in prev)) else mark.append(1)
        elif sym == '-':
            mark.append(0) if (('*' in prev) or ('/' in prev)) else mark.append(1)
        else:
            mark.append(1)

        
        
        prev = step
    
    
    
    
    
    return (sol,mark)

In [6]:
test = generateEquation()
(sol,mark) = generateStep(test)
for step in sol:
    print(len(step))

9
9
9
9
9


In [7]:
def Token(a):
    _a = []
    for x in range(2):
        _step = []
        for i in a[x]:
            if i == '':
                _i=0
            else:
                try:
                    _i = float(i)
                except:
                    _i = symbol[i]
            _step.append(_i)
        _a.append(_step)
    return _a

In [8]:
size = 2000

traindata=[]
trainlabel=[]

for i in range(size):
    try:
        (sol, mark) = generateStep(generateEquation())
        for j in range(4):
            traindata.append(Token(sol[j:j+2]))
        trainlabel+=mark
    except:
        i=i-1

print(len(traindata),len(trainlabel))
#print(traindata)
#print('\n')
#print(trainlabel)

7940 7940


In [9]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(36, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(2, activation=tf.nn.softmax)])

model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(traindata, trainlabel, epochs=10)

Epoch 1/10
249/249 [==============================] - 0s 469us/step - loss: 0.9205 - accuracy: 0.7202
Epoch 2/10
249/249 [==============================] - 0s 473us/step - loss: 0.5040 - accuracy: 0.7790
Epoch 3/10
249/249 [==============================] - 0s 457us/step - loss: 0.3910 - accuracy: 0.8385
Epoch 4/10
249/249 [==============================] - 0s 481us/step - loss: 0.3419 - accuracy: 0.8686
Epoch 5/10
249/249 [==============================] - 0s 481us/step - loss: 0.2896 - accuracy: 0.8975
Epoch 6/10
249/249 [==============================] - 0s 461us/step - loss: 0.2805 - accuracy: 0.9072
Epoch 7/10
249/249 [==============================] - 0s 473us/step - loss: 0.2367 - accuracy: 0.9170
Epoch 8/10
249/249 [==============================] - 0s 465us/step - loss: 0.2114 - accuracy: 0.9300
Epoch 9/10
249/249 [==============================] - 0s 469us/step - loss: 0.2108 - accuracy: 0.9252
Epoch 10/10
249/249 [==============================] - 0s 477us/step - loss: 0.222

In [11]:
test_size = 1000

testdata=[]
testlabel=[]

for i in range(size):
    try:
        (sol, mark) = generateStep(generateEquation())
        for j in range(4):
            testdata.append(Token(sol[j:j+2]))
        testlabel+=mark
    except:
        i=i-1

In [12]:
model.evaluate(testdata, testlabel)

249/249 [==============================] - 0s 380us/step - loss: 0.2404 - accuracy: 0.9293


[0.24040469527244568, 0.9292713403701782]

In [14]:
#Standardizer
LENGTH = 9
def Standardize(a):
    sol = []
    for line in a:
        step = []
        l = len(line)
        
        start = 0
        end = 0
        e = 0
        while end < l:
            while line[end] not in symbol.keys():
                end+=1
                if end >= l:
                    break
            step.append(line[start:end])
            
            if end >= l:
                break
                
            sym = line[end]
            if len(sol) > 0:
                e = len(step)+1
                while sym != sol[0][e]:
                    step.append('')
                    e+=1
                
            step.append(sym)
            
            start=end+1
            end+=1
        
        step += ['']*(LENGTH-len(step))   
        sol.append(step)
    
    return sol

In [15]:
#Practice

formula = "1+2*3-8/4 = 3*3-8/4 = 3*3-2 = 9-2 = 7"
lines = formula.split("=")
for i in range(len(lines)):
    lines[i]=lines[i].strip()

sol = Standardize(lines)
n = len(sol)

print(''.join(sol[0]))

for j in range(1, n-1):
    input_sol = Token(sol[j-1:j+1])
    result = model.predict([input_sol])
    print('=',''.join(sol[j]), end=' ')
    print("\t\t",np.argmax(result),round(max(result[0])*100,2))

print('=',''.join(sol[n-1]))

        




1+2*3-8/4
= 3*3-8/4 		 0 100.0
= 3*3-2 		 1 100.0
= 9-2 		 1 99.3
= 7


In [22]:
(sol, mark) = generateStep(generateEquation())

print(''.join(sol[0]))
for j in range(1,4):
    input_sol = Token(sol[j-1:j+1])
    result = model.predict([input_sol])
    print('=',''.join(sol[j]), end=' ')
    print("\t\t",np.argmax(result),round(max(result[0])*100,2))

print('=',''.join(sol[n-1]))

8*8+1*10*10
= 64+1*10*10 		 1 99.8
= 65*10*10 		 0 89.2
= 650*10 		 1 100.0
= 6500


In [25]:
#export model

# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

!mkdir -p saved_model
model.save('saved_model/my_model') 

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file saved_model already exists.
Error occurred while processing: saved_model.


INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [ ]:
# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

In [26]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 18)                0         
_________________________________________________________________
dense (Dense)                (None, 36)                684       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 74        
Total params: 758
Trainable params: 758
Non-trainable params: 0
_________________________________________________________________


In [27]:
(sol, mark) = generateStep(generateEquation())

print(''.join(sol[0]))
for j in range(1,4):
    input_sol = Token(sol[j-1:j+1])
    result = new_model.predict([input_sol])
    print('=',''.join(sol[j]), end=' ')
    print("\t\t",np.argmax(result),round(max(result[0])*100,2))

print('=',''.join(sol[n-1]))

10/9+9/4+7
= 10/9+9/11 		 0 98.02
= 10/18/11 		 0 98.31
= 10/1.6363636363636365 		 1 97.93
= 6.111111111111111
